# Reading and Plotting Archive Surface Data

This notebook demonstrates reading surface data from the Valpo archive (1950-2018).

### Import Packages

In [ ]:
from datetime import datetime, timedelta

from metpy.plots.declarative import *
from metpy.units import units
import pandas as pd

### Get Data

Using Pandas package to read in remote CSV files. Note that this data doesn't have any units associated with it. You'll need to assume the standard units that come with METAR files, which means that temperature is in Celsius, pressure is in hPa, and winds are in knots.

In [ ]:
date = datetime(2011, 2, 1, 0)

dt = timedelta(minutes=15)
sdate = date - dt
edate = date + dt
data_url = ('http://mesonet.agron.iastate.edu/cgi-bin/request/asos.py?data=all&tz=Etc/UTC&format=comma&latlon=yes&'
            f'year1={sdate.year}&month1={sdate.month}&day1={sdate.day}&hour1={sdate.hour}&minute1={sdate.minute}&'
            f'year2={edate.year}&month2={edate.month}&day2={edate.day}&hour2={edate.hour}&minute2={edate.minute}')
data = pd.read_csv(data_url, skiprows=5, na_values=['M'], low_memory=False).replace('T', 0.00001).groupby('station').tail(1)
df = metar.parse_metar_file(StringIO('\n'.join(val for val in data.metar)), year=date.year, month=date.month)

In [ ]:
list(df.keys())

### Plot Data

In [ ]:
mslp_formatter = lambda v: format(v*10, '.0f')[2:]
df['tmpf'] = (df.air_temperature.values * units.degC).to('degF')
df['dwpf'] = (df.dew_point_temperature.values * units.degC).to('degF')


# Plot desired data
obs = PlotObs()
obs.data = df
obs.time = date
obs.time_window = timedelta(minutes=15)
obs.level = None
obs.fields = ['cloud_coverage', 'tmpf', 'dwpf',
              'air_pressure_at_sea_level', 'present_weather']
obs.locations = ['C', 'NW', 'SW', 'NE', 'W']
obs.formats = ['sky_cover', None, None, mslp_formatter, 'current_weather']
obs.reduce_points = 0.75
obs.vector_field = ['eastward_wind', 'northward_wind']

# Panel for plot with Map features
panel = MapPanel()
panel.layout = (1, 1, 1)
panel.projection = 'lcc'
panel.area = 'in'
panel.layers = ['states']
panel.plots = [obs]

# Bringing it all together
pc = PanelContainer()
pc.size = (10, 10)
pc.panels = [panel]

pc.show()